# Corn, wheat, oats, and rye yield in Argentina (kg/ha)
##    Period: 1923-2020

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wg
from ipywidgets import interact, interactive, IntSlider
pd.options.display.float_format = "{:.2f}".format        # floating point with 2 decimal places
import warnings
warnings.filterwarnings('ignore')

lst_crops = ['maiz', 'trigo', 'avena' , 'centeno']
lst_col = ['Corn', 'Wheat', 'Oats' , 'Rye']
colors = ['#FF8000', '#008080', '#835B0A', '#8d00ff']

# clean_df
This function is responsible for data analysis and cleaning tasks.

In [ ]:
def clean_df(crops) :
    lst_df= []
    for crop in crops:  
        if crop == 'avena' or crop == 'centeno' or crop =='maiz':
            if crop =='maiz' :
                df = pd.read_csv('assets/' + crop + "-serie-1923-2019" + ".csv", 
                             encoding = "latin-1", usecols=[1,10])
            else :
                df = pd.read_csv('assets/' + crop + "-serie-1923-2020" + ".csv", 
                                 encoding = "latin-1", usecols=[1,10])
            df.rename(columns={'anio': 'year', 'rendimiento_kgxha': crop}, inplace=True)
        elif crop == 'trigo':
            df = pd.read_csv('assets/' + crop + "-serie-1923-2020-anual" + ".csv",
                             encoding = "latin-1", usecols=[0,4])
            df.rename(columns={'indice_tiempo': 'year', 'rendimiento_trigo_kgxha': crop}, 
                      inplace=True)
           
    # pd.options.display.float_format = "{:.2f}".format  : floating point with 2 decimal places
    # WARNING: .applymap('{:,.2f}'.format)               : do not use converts to string
    # Group by year, average calculation
        df1 = df.groupby('year').agg({crop:np.nanmean})
        lst_df.append(df1)
          
    # Concat lists.
    df_crops = pd.concat(lst_df, axis=1)

    # The 2020 Corn value is missing, so the 2019 value is repeated.
    df_crops = df_crops.fillna(method='ffill')
    
    return df_crops

# figure_axis
This function is responsible for plotting the figure, creating the title, legend, axis labels, and horizontal lines.

In [ ]:
def figure_axis():
    fig, ax = plt.subplots(1,1, figsize = (11,6), facecolor='#4D88FF')     
    
    # Range axis
    plt.gca().axis([1920,2021,-400,7500])
    
    # disable the entire frame
    plt.gca().spines[['top','bottom','left','right']].set_visible(False)

    # Title
    plt.title('Argentine Republic\n Period 1923-2020',color = 'white', 
              fontsize = 12, fontstyle= 'normal', fontweight  = 'heavy')
    
    # Legend center text in x=1933  y=4000
    plt.text(1933, 4000, "Average yield per year\nkg/ha: kilogram per hectare", 
             ha= 'center', va='center', fontsize=8, fontweight  = 'bold', 
             color = 'yellow', backgroundcolor = '#4D88FF')  
    
    # Labels both axis
    plt.xlabel('Year', fontweight = 'bold', color = '#98afb4')
    plt.ylabel('Average yield kg/ha', fontweight = 'bold', color = '#98afb4')
        
    # set color 
    ax.set_facecolor('white')
    # tick params both axis
    ax.tick_params(color='#98afb4', labelcolor='white', labelsize=9, bottom=True)
    
    # Ticks axis x : tick year for each 10 years
    year= np.arange(1923,2021,10)
    ax.set_xticks(ticks = year, labels = [str(x) for x in year])
    
    # Plot horizontal lines
    lines = [0,1000,3000,5000,7000]
    for i in lines:
        plt.axhline(y=i, xmin = 0.01, xmax = 0.98, color='#4D88FF', linestyle=":")

    return fig,ax

# plot_performance
This function creates four line plots, one for each crop.

In [ ]:
def plot_performance(crops, lst_col, pal_colors):
    fig,ax = figure_axis()
    
    # line plot for the 4 crops
    for n in range(len(crops)):
        df = clean_df(crops)[crops[n]]
        ax.plot(df, '-', color = pal_colors[n])
        
        #  df.index[-1]+1 = last year    ,     df.iloc[-1]= value of each crop axis y
        x , y = df.index[-1]+1, df.iloc[-1]
        ax.text(x, y, lst_col[n], color = pal_colors[n], fontweight  = 'bold')

# change_year
This function displays the yield in kilograms per hectare (kg/ha) for each crop according to the selected year. For years less than or equal to 1972, the text is displayed to the right of the vertical bar. For years greater than 1972, the text is displayed to the left.  The selected year is highlighted at the bottom of the bar.

In [ ]:
def change_year(year_input):
        
    df = clean_df(lst_crops)
    plot_performance(lst_crops, lst_col, colors)
      
    # Update the position of the vertical line and text based on the year.
    plt.axvline(year_input, ymin = 0.05, ymax = 0.98, color = "C0")
    plt.text(year_input, 300 , year_input, fontsize=8, ha='center',
             fontweight  = 'bold', color = "white", backgroundcolor = "C0")        
    
    # The crop text and its respective yield (2 decimal places).
    s = ''
    for n in range(len(lst_crops)):
        s = s + lst_col[n] + ' : '+  "{:.2f}".format(df.loc[year_input][n])+ '\n'
    
    # The default year is 1972.
    if year_input <= 1972:
        plt.text(year_input, 6500 , s, fontsize=6, ha='left', 
                 fontweight  = 'bold', color = '#6d848a', ma='left')  
    else:
        plt.text(year_input, 6500 , s, fontsize=6, ha='right', 
                 fontweight  = 'bold', color = '#6d848a', ma='left')  
    
    plt.show()

**First, the widget is defined. Then, the change_function() function is called.**

In [ ]:
x_entrada = wg.IntSlider(min=1923, max=2020, step=1, value = 1972, description=' ', 
                        orientation='horizontal', continuous_update=False, 
                        layout=wg.Layout(width='99%'))

widget=wg.interact(change_year, year_input = x_entrada)